In [ ]:
KEYS = {
    "ts": "$f_{\\text{ts}}$",
    "wiki": "$f_{\\text{wiki}}$",
    "ftsqts": "$f_{\\text{ts}} q_{\\text{ts}}$",
    "fwikiqwiki": "$f_{\\text{wiki}} q_{\\text{wiki}}$",
    "ftsqwiki": "$f_{\\text{ts}} q_{\\text{wiki}}$",
    "fwikiqts": "$f_{\\text{wiki}} q_{\\text{ts}}$",
}
prefix = "eval/mean_log_prob/"

import pandas as pd 
import wandb
api = wandb.Api()

runs = api.runs("[REDACTED]")

history_list, config_list, name_list = [], [], []
for run in runs:
    history_list.append(run.history(keys=[prefix + k for k in KEYS]))

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "history": history_list,
    "name": name_list
})

In [11]:
def parse_args(run_name):
    _ = run_name[run_name.find("occur")+len("occur"):]
    _ = _.split("_")
    in_occur = int(_[0])

    _ = run_name[run_name.find("dataseed")+len("dataseed"):]
    _ = _[_.find("seed")+len("seed"):]
    seed = int(_.split("_")[0])

    _ = run_name[run_name.find("dataseed")+len("dataseed"):]
    dataseed = int(_.split("_")[0])

    _ = run_name[run_name.find("overlap")+len("overlap"):]
    overlap = int(_.split("_")[0])

    return {
        "in_occur": in_occur,
        "seed": seed,
        "dataseed": dataseed,
        "overlap": overlap,
    }

In [ ]:
import os
import yaml
import json
import matplotlib.pyplot as plt
import numpy as np

def gather_data(runs_df, overlap):
    plot_data = {k: {} for k in KEYS}
    
    for i in range(len(runs_df.index)):
        run_data = runs_df.iloc[i]["history"]

        try:
            args = parse_args(runs_df.iloc[i]["name"])
        except:
            continue
        
        if args["overlap"] != overlap:
            continue

        seed = args["seed"]
        
        for key in KEYS:
            plot_data[key][seed] = run_data[prefix + key].tail(10).mean()

    data = {}
    for key in KEYS:
        data[key] = np.mean(list(plot_data[key].values()))
    
    return data

print(gather_data(runs_df, 0))
print(gather_data(runs_df, 1))
